# Crystallography

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

In [ ]:
from materialite import (
    Orientation,
    Vector,
    Order4SymmetricTensor,
    SlipSystem,
    add_ipf_colors_field,
    get_ipf,
    import_dream3d,
)

import matplotlib.pyplot as plt
import numpy as np

Import data from DREAM.3D and visualize the grain structure.

In [ ]:
data_container = "DataContainers/SyntheticVolumeDataContainer"
material = import_dream3d(
    file="ipf_example/example_for_ipf_map.dream3d",
    simpl_geometry_path=f"{data_container}/_SIMPL_GEOMETRY",
    region_id_path=f"{data_container}/EBSD Scan Data/FeatureIds",
    region_field_paths=[
        f"{data_container}/Grain Data/EulerAngles",
    ],
)
material.plot("feature_ids")

Create orientations using the pointwise Euler angles imported from DREAM.3D, and add them as a field to the `Material`. Materialite uses the same Euler angle convention as DREAM.3D, specifically the Bunge Euler angles representing a basis transformation from the specimen reference frame to the crystal reference frame.

In [ ]:
eulers = material.extract(["euler_angles_1", "euler_angles_2", "euler_angles_3"])
orientations = Orientation.from_euler_angles(eulers)
material = material.create_fields({"orientation": orientations})

Use the orientations to compute the maximum Schmid factor at each point, assuming an FCC material (i.e., with the octahedral slip systems) and loading in the specimen "z" direction. We can do this by representing the loading direction within the reference frames of the crystals and using the `max_schmid_factor` method of the slip systems.

In [ ]:
loading_direction = Vector([0.0, 0.0, 1.0])
slip_systems = SlipSystem.octahedral()
crystal_loading_directions = loading_direction.to_crystal_frame(orientations)
max_schmid_factors = slip_systems.max_schmid_factor(crystal_loading_directions)

We could also calculate the Schmid factors by represent the Schmid tensors in the specimen reference frame. Here, we explicitly do the calculation $m_{max} = \mathbf{M} : (\mathbf{d} \otimes \mathbf{d})$, where $m_{max}$ is the max Schmid factor in a crystal, $\mathbf{M}$ is the crystal Schmid tensor, and $\mathbf{d}$ is the loading direction.

In [ ]:
specimen_schmid_tensors = slip_systems.schmid_tensor.to_specimen_frame(orientations)
max_schmid_factors2 = (
    specimen_schmid_tensors * loading_direction.outer(loading_direction)
).abs.max("s")

Verify that the two calculations match.

In [ ]:
np.allclose(max_schmid_factors.components, max_schmid_factors2.components)

Use the orientations to compute the stiffness in the loading direction at each point.

In [ ]:
stiffness_tensor = Order4SymmetricTensor.from_cubic_constants(C11=250, C12=160, C44=120)
directional_moduli = stiffness_tensor.directional_modulus(crystal_loading_directions)

Again, we could also do this calculation in the specimen frame instead.

In [ ]:
specimen_frame_stiffness_tensors = stiffness_tensor.to_specimen_frame(orientations)
directional_moduli2 = specimen_frame_stiffness_tensors.directional_modulus(
    loading_direction
)
np.allclose(directional_moduli.components, directional_moduli2.components)

Add fields to the material for the max Schmid factor, the directional modulus, and an IPF map with respect to the loading direction. The function `add_ipf_colors_field` requires the name of the field containing the `Orientation` of each point, the loading direction (as a list, numpy array, or Materialite `Vector`), and the type of unit cell (cubic or hexagonal).

In [ ]:
material = material.create_fields(
    {
        "orientation": orientations,
        "max_schmid_factor": max_schmid_factors,
        "directional_modulus": directional_moduli,
    }
).run(
    add_ipf_colors_field,
    orientation_label="orientation",
    specimen_frame_direction=loading_direction,
    unit_cell="cubic",
)

Plot the new fields that we created. This turns out to be an interesting case since there's one grain that deviates from the fairly strong texture.

In [ ]:
material.plot("directional_modulus")

In [ ]:
material.plot("max_schmid_factor")

In [ ]:
material.plot("ipf_color", kind="ipf_map")

Here is the IPF map legend:

In [ ]:
from PIL import Image
import numpy as np

ipf_legend = np.asarray(Image.open("ipf_example/ipf_map_legend_cubic.png"))
fig, ax = plt.subplots()
ax.imshow(ipf_legend)
ax.axis("off")

Make an inverse pole figure on the standard stereographic triangle. The `get_ipf` function takes the same inputs as `add_ipf_colors_field`. It returns the point on the IPF for each point in the `Material` and the boundary of the SST so that we can make a scatterplot of the IPF.

In [ ]:
ipf_points, ipf_boundary = get_ipf(
    specimen_frame_direction=loading_direction,
    orientations=orientations,
    unit_cell="cubic",
)
fig, ax = plt.subplots()
ax.plot(ipf_boundary[:, 0], ipf_boundary[:, 1], "k")
ax.scatter(ipf_points[:, 0], ipf_points[:, 1])
# labels for crystal directions:
ax.text(
    -0.05,
    -0.01,
    "$[001]$",
    horizontalalignment="left",
    verticalalignment="top",
    fontsize=36,
)
ax.text(
    0.35,
    -0.01,
    "$[101]$",
    horizontalalignment="left",
    verticalalignment="top",
    fontsize=36,
)
ax.text(
    0.3,
    0.37,
    "$[111]$",
    horizontalalignment="left",
    verticalalignment="bottom",
    fontsize=36,
)
ax.set_aspect(1.0)
ax.axis("off")

Do the same thing but with a hexagonal unit cell and the basal and prismatic slip systems

In [ ]:
material = (
    material.remove_field("ipf_color")
    .remove_field("directional_modulus")
    .remove_field("max_schmid_factor")
)

In [ ]:
slip_systems = SlipSystem.basal().concatenate(slip_systems.prismatic())
max_schmid_factors = slip_systems.max_schmid_factor(crystal_loading_directions)

stiffness_tensor = Order4SymmetricTensor.from_transverse_isotropic_constants(
    C11=160, C12=90, C13=70, C33=180, C44=45
)
directional_moduli = stiffness_tensor.directional_modulus(crystal_loading_directions)

In [ ]:
material = material.create_fields(
    {
        "max_schmid_factor": max_schmid_factors,
        "directional_modulus": directional_moduli,
    }
).run(
    add_ipf_colors_field,
    orientation_label="orientation",
    specimen_frame_direction=loading_direction,
    unit_cell="hexagonal",
)

In [ ]:
material.plot("directional_modulus")

In [ ]:
material.plot("max_schmid_factor")

In [ ]:
material.plot("ipf_color", kind="ipf_map")

In [ ]:
ipf_legend = np.asarray(Image.open("ipf_example/ipf_map_legend_hcp.png"))
fig, ax = plt.subplots()
ax.imshow(ipf_legend)
ax.axis("off")

In [ ]:
ipf_points, ipf_boundary = get_ipf(
    specimen_frame_direction=loading_direction,
    orientations=orientations,
    unit_cell="hexagonal",
)
fig, ax = plt.subplots()
ax.plot(ipf_boundary[:, 0], ipf_boundary[:, 1], "k")
ax.scatter(ipf_points[:, 0], ipf_points[:, 1])
ax.text(
    -0.15,
    -0.02,
    "$[0001]$",
    horizontalalignment="left",
    verticalalignment="top",
    fontsize=36,
)
ax.text(
    0.8,
    -0.02,
    "$[2\\bar{1}\\bar{1}0]$",
    horizontalalignment="left",
    verticalalignment="top",
    fontsize=36,
)
ax.text(
    0.65,
    0.5,
    "$[10\\bar{1}0]$",
    horizontalalignment="left",
    verticalalignment="bottom",
    fontsize=36,
)
ax.set_aspect(1.0)
ax.axis("off")